# 21. Feature Engineering

Everything has been super disappointing up to this point.

In [1]:
import sys
from typing import Tuple
import logging

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pycaret.classification import *

## Configure Logging

In [2]:
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [3]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_06.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2023-03-08 16:35:01 INFO     Started


## Load Source Data

In [4]:
# load the source training set
df_source = pd.read_csv('data/train.csv.gz', compression='gzip', index_col=1)

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

(13174211, 19)


,session_id,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
index,,,,,,,,,,,,,,,,,,,
0,20090312431273200,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [5]:
# load the source training labels
df_source_labels = pd.read_csv('data/train_labels.csv')

print(df_source_labels.shape)
with pd.option_context('display.max_columns', None):
    display(df_source_labels.head(3))

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Functions

### Functions to prepare the data

In [6]:
def map_question_to_level_group(question_number):
    """
    Maps the question number to the level group.

    Parameters
    ----------
    question_number : int
        The question number.

    Returns
    -------
    str
        The level group.
    """
    if question_number in [1, 2, 3]:
        return '0-4'
    elif question_number in [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return '5-12'
    elif question_number in [14, 15, 16, 17, 18]:
        return '13-22'
    else:
        return None

def prepare_label_dataset(data : pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the label dataset and add columns for the level group 
    and the question number.

    Parameters
    ----------
    data : pd.DataFrame
        The label dataset.

    Returns
    -------
    pd.DataFrame
        The prepared label dataset.
    """
    # add the columns to determine the level group
    df_labels = data \
        .rename(columns={'session_id': 'id'}) \
        .assign(session_id=lambda df: df['id'].str.split('_').str[0].astype(int)) \
        .assign(question_id=lambda df: df['id'].str.split('_').str[1]) \
        .assign(question_num=lambda df: df['question_id'].str[1:].astype(int)) \
        [['session_id', 'question_num', 'correct']]
    
    # add the level group column
    df_labels['level_group'] = df_labels['question_num'].apply(map_question_to_level_group) 
        
    return df_labels

In [7]:
def prepare_main_dataset(data : pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the main dataset by removing duplicates and removing 
    columns that are not needed.

    Parameters
    ----------
    data : pd.DataFrame
        The main dataset.

    Returns
    -------
    pd.DataFrame
        The prepared main dataset.
    """
    empty_columns = ['fullscreen', 'hq', 'music', 'page', 'hover_duration']

    df_main = data \
        .drop_duplicates() \
        .reset_index(drop=True) \
        .drop(empty_columns, axis=1) \
        .drop('text', axis=1)

    return df_main

In [8]:
def select_sessions(
        y: pd.DataFrame,
        sample_size: int,
        random_state: int=1337) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Selects a sample of sessions from the dataset.

    Parameters
    ----------
    y : pd.DataFrame
        The label dataset.
    sample_size : int
        The number of sessions to select.
    random_state : int
        The random state to use.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        The selected session ids, the main dataset and the label dataset.
    """
    # select all the unique session ids
    all_session_ids = y['session_id'].unique()

    # create a sample for testing
    session_ids = np.random.choice(all_session_ids, size=sample_size, replace=False)

    # split the dataset into train, validation and test sets
    train, test = train_test_split(session_ids, test_size=0.4)
    test, val = train_test_split(test, test_size=0.5)

    # print the number of sessions in each set
    print(f'Train: {len(train)}')
    print(f'Validation: {len(val)}')
    print(f'Test: {len(test)}')

    return train, val, test

## Data Preprocessing

In [9]:
# prepare the main dataset
df_source = prepare_main_dataset(df_source)

with pd.option_context('display.max_columns', None):
    print(df_source.shape)
    display(df_source.head(3))  

(13173445, 13)


,session_id,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,fqid,room_fqid,text_fqid,level_group
0,20090312431273200,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4
1,20090312431273200,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
2,20090312431273200,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4


In [10]:
# prepare the label dataset
df_source_labels = prepare_label_dataset(df_source_labels)

with pd.option_context('display.max_columns', None):
    display(df_source_labels.sample(n=3, random_state=51))

,session_id,question_num,correct,level_group
21476,22010116250792520,2,1,0-4
84068,21000111433937450,8,1,5-12
171219,21040510125933256,15,0,13-22


## Data Selection

In [11]:
random_state = 51
sample_size = df_source_labels['session_id'].nunique()
#sample_size=2000

In [12]:
train, val, test = select_sessions(
    y=df_source_labels,
    sample_size=sample_size,
    random_state=random_state)

Train: 7067
Validation: 2356
Test: 2356


## Functions for Modeling Data

### Create Basic Dataset

In [13]:
def create_basic_session_features(df_session:pd.DataFrame,
                                  level_group:str) -> pd.DataFrame:
    """
    Create a dataset with the most basic features for a single session.

    Parameters
    ----------
    df_session : pd.DataFrame
        The session dataset.

    Returns
    -------
    pd.DataFrame
        The session dataset.
    """
    df_features = df_session \
        .query(f'level_group == "{level_group}"') \
        [['level_group']] \
        .drop_duplicates() \
        .reset_index(drop=True)

    return df_features

# test the function
session_id = train[0]
df_session = df_source[df_source['session_id'] == session_id]

df_session = create_basic_session_features(df_session, '0-4')
print(df_session.shape)
with pd.option_context('display.max_columns', None):
    display(df_session.tail(3))

(1, 1)


,level_group
0,0-4


### Generic functions for creating feature datasets

In [14]:
def create_session_dataset(X:pd.DataFrame,
                                 y:pd.DataFrame,
                                 session_id:str,
                                 create_features) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Create a dataset with the most basic features for a single session.

    Parameters
    ----------
    X : pd.DataFrame
        The main dataset.
    y : pd.DataFrame
        The label dataset.
    session_id : str
        The session id.

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        The session dataset and the session label dataset.
    """
    features = pd.DataFrame()
    y_true = []

    # select the session data
    df_session_labels = y.query('session_id == @session_id')
    df_session = X.query('session_id == @session_id')

    # create the features for each level group
    features_group = {
        '0-4': create_features(df_session, '0-4'),
        '5-12': create_features(df_session, '5-12'),
        '13-22': create_features(df_session, '13-22'),
    }

    # iterate over all the questions answered in the session
    for _, row in df_session_labels.iterrows():
        # get the question number, correct answer and level group
        question_num = row['question_num']
        correct = row['correct']
        level_group = row['level_group']

        # append the label as the target value
        y_true.append(correct)

        # get the features for the level group
        question_features = features_group[level_group] \
            .assign(question_num=question_num) \
            .drop('level_group', axis=1)
        
        # append the features to the dataset
        features = pd.concat([features, question_features], axis=0)


    return features.reset_index(drop=True), np.array(y_true)

# test the function
session_id = train[0]
df_session, y_session = create_session_dataset(
    X=df_source,
    y=df_source_labels,
    session_id=session_id,
    create_features=create_basic_session_features)

df_session['correct'] = y_session

print(df_session.shape)
with pd.option_context('display.max_columns', None):
    display(df_session.head(3))

(18, 2)


,question_num,correct
0,1,0
1,2,1
2,3,1


In [15]:
def create_dataset(X:pd.DataFrame,
                   y:pd.DataFrame,
                   session_list:np.ndarray,
                   create_features) -> pd.DataFrame:
    
    df_sessions = pd.DataFrame()
    
    for session_id in tqdm(session_list):
        # create the session dataset
        df_session, y_session = create_session_dataset(
            X=X,
            y=y,
            session_id=session_id,
            create_features=create_features)
        
        # add the label for pycaret training
        df_session['correct'] = y_session

        # append the session dataset to the main dataset
        df_sessions = pd.concat([df_sessions, df_session], axis=0)

    return df_sessions.reset_index(drop=True)


# test the function
df_basic = create_dataset(
    X=df_source,
    y=df_source_labels,
    session_list=train[:3],
    create_features=create_basic_session_features)

print(df_basic.shape)
with pd.option_context('display.max_columns', None):
    display(df_basic.head(3))

  0%|          | 0/3 [00:00<?, ?it/s]

(54, 2)


,question_num,correct
0,1,0
1,2,1
2,3,1


### Elapsed Time Features

In [16]:
def create_elapsed_time_session_features(df_session:pd.DataFrame,
                                  level_group:str) -> pd.DataFrame:
    """
    Create a dataset with the most basic features for a single session.

    Parameters
    ----------
    df_session : pd.DataFrame
        The session dataset.

    Returns
    -------
    pd.DataFrame
        The session dataset.
    """
    df_features = df_session \
        .query(f'level_group == "{level_group}"') \
        .groupby('level_group') \
        .agg({'elapsed_time': 'max'}) \
        .reset_index(drop=False)

    return df_features

# test the function
session_id = train[0]
df_session = df_source[df_source['session_id'] == session_id]

df_session = create_elapsed_time_session_features(df_session, '0-4')
print(df_session.shape)
with pd.option_context('display.max_columns', None):
    display(df_session.tail(3))

(1, 2)


,level_group,elapsed_time
0,0-4,114238


### Event Count Features

In [17]:
def create_event_count_session_features(df_session:pd.DataFrame,
                                  level_group:str) -> pd.DataFrame:
    """
    Create a dataset with the event features for a single session.

    Parameters
    ----------
    df_session : pd.DataFrame
        The session dataset.

    Returns
    -------
    pd.DataFrame
        The session dataset.
    """
    df_features = df_session \
        .query(f'level_group == "{level_group}"') \
        .groupby('event_name') \
        .agg({'event_name': ['count']}) \
        .T \
        .assign(level_group=level_group) \
        .reset_index(drop=True)

    # df_features = df_session \
    #     .query(f'level_group == "{level_group}"') \
    #     .groupby('level_group') \
    #     .agg({'elapsed_time': 'max'}) \
    #     .reset_index(drop=False)

    return df_features

# test the function
session_id = train[0]
df_session = df_source[df_source['session_id'] == session_id]

df_session = create_event_count_session_features(df_session, '0-4')
print(df_session.shape)
with pd.option_context('display.max_columns', None):
    display(df_session.head(10))

(1, 11)


event_name,checkpoint,cutscene_click,map_click,map_hover,navigate_click,notification_click,object_click,object_hover,observation_click,person_click,level_group
0,1,47,2,3,80,9,15,5,2,22,0-4


## Model Training

In [18]:
def test_features(X:pd.DataFrame,
                  y:pd.DataFrame,
                  create_features) -> pd.DataFrame:
    
    # create the datasets
    df_train = create_dataset(X=X, y=y, session_list=train, create_features=create_features)
    df_val = create_dataset(X=X, y=y, session_list=val, create_features=create_features)
    df_test = create_dataset(X=X, y=y, session_list=test, create_features=create_features)

    # prepare the classifier
    classifier = setup(
        data=df_train,
        test_data=df_val,
        target='correct',
        train_size=1,
        session_id=random_state,
        fix_imbalance=False,
        fix_imbalance_method='RandomOverSampler',
        html=False,
        verbose=True)
    
    # compare the models
    top_model = compare_models(exclude=['knn'], n_select=5)

    # test the top model
    df_predicted = predict_model(estimator=top_model[0], data=df_test)
    print(classification_report(y_true=df_predicted.correct, y_pred=df_predicted.prediction_label))    

### Basic Dataset

In [19]:
test_features(
    X=df_source,
    y=df_source_labels,
    create_features=create_basic_session_features)

  0%|          | 0/7067 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

                    Description             Value
0                    Session id                51
1                        Target           correct
2                   Target type            Binary
3           Original data shape       (169614, 2)
4        Transformed data shape       (169614, 2)
5   Transformed train set shape       (127206, 2)
6    Transformed test set shape        (42408, 2)
7              Numeric features                 1
8                    Preprocess              True
9               Imputation type            simple
10           Numeric imputation              mean
11       Categorical imputation              mode
12               Fold Generator   StratifiedKFold
13                  Fold Number                10
14                     CPU Jobs                -1
15                      Use GPU             False
16               Log Experiment             False
17              Experiment Name  clf-default-name
18                          USI              f147


                                    Model  Accuracy     AUC  Recall   Prec.  \
dt               Decision Tree Classifier    0.7329  0.7296  0.9407  0.7466   
rf               Random Forest Classifier    0.7329  0.7296  0.9407  0.7466   
gbc          Gradient Boosting Classifier    0.7329  0.7291  0.9407  0.7466   
et                 Extra Trees Classifier    0.7329  0.7296  0.9407  0.7466   
xgboost         Extreme Gradient Boosting    0.7329  0.7296  0.9407  0.7466   
lightgbm  Light Gradient Boosting Machine    0.7329  0.7296  0.9407  0.7466   
ada                  Ada Boost Classifier    0.7308  0.7217  0.9786  0.7310   
lr                    Logistic Regression    0.7055  0.5593  1.0000  0.7055   
nb                            Naive Bayes    0.7055  0.6234  1.0000  0.7055   
svm                   SVM - Linear Kernel    0.7055  0.0000  1.0000  0.7055   
ridge                    Ridge Classifier    0.7055  0.0000  1.0000  0.7055   
qda       Quadratic Discriminant Analysis    0.7055 

### Elapsed Time Dataset

In [20]:
test_features(
    X=df_source,
    y=df_source_labels,
    create_features=create_elapsed_time_session_features)

  0%|          | 0/7067 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

                    Description             Value
0                    Session id                51
1                        Target           correct
2                   Target type            Binary
3           Original data shape       (169614, 3)
4        Transformed data shape       (169614, 3)
5   Transformed train set shape       (127206, 3)
6    Transformed test set shape        (42408, 3)
7              Numeric features                 2
8                    Preprocess              True
9               Imputation type            simple
10           Numeric imputation              mean
11       Categorical imputation              mode
12               Fold Generator   StratifiedKFold
13                  Fold Number                10
14                     CPU Jobs                -1
15                      Use GPU             False
16               Log Experiment             False
17              Experiment Name  clf-default-name
18                          USI              0022


                                    Model  Accuracy     AUC  Recall   Prec.  \
gbc          Gradient Boosting Classifier    0.7375  0.7386  0.9451  0.7487   
lightgbm  Light Gradient Boosting Machine    0.7371  0.7383  0.9392  0.7507   
xgboost         Extreme Gradient Boosting    0.7346  0.7322  0.9310  0.7519   
ada                  Ada Boost Classifier    0.7309  0.7290  0.9759  0.7320   
lr                    Logistic Regression    0.7055  0.4272  1.0000  0.7055   
ridge                    Ridge Classifier    0.7055  0.0000  1.0000  0.7055   
lda          Linear Discriminant Analysis    0.7055  0.5622  1.0000  0.7055   
dummy                    Dummy Classifier    0.7055  0.5000  1.0000  0.7055   
nb                            Naive Bayes    0.7015  0.4627  0.9889  0.7059   
qda       Quadratic Discriminant Analysis    0.7014  0.6187  0.9884  0.7060   
rf               Random Forest Classifier    0.6490  0.6481  0.7508  0.7514   
dt               Decision Tree Classifier    0.6484 

### Event Count

In [21]:
test_features(
    X=df_source,
    y=df_source_labels,
    create_features=create_event_count_session_features)

  0%|          | 0/7067 [00:00<?, ?it/s]

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


  0%|          | 0/2356 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

                    Description             Value
0                    Session id                51
1                        Target           correct
2                   Target type            Binary
3           Original data shape      (169614, 13)
4        Transformed data shape      (169614, 13)
5   Transformed train set shape      (127206, 13)
6    Transformed test set shape       (42408, 13)
7              Numeric features                12
8      Rows with missing values             38.4%
9                    Preprocess              True
10              Imputation type            simple
11           Numeric imputation              mean
12       Categorical imputation              mode
13               Fold Generator   StratifiedKFold
14                  Fold Number                10
15                     CPU Jobs                -1
16                      Use GPU             False
17               Log Experiment             False
18              Experiment Name  clf-default-name


                                    Model  Accuracy     AUC  Recall   Prec.  \
lightgbm  Light Gradient Boosting Machine    0.7489  0.7648  0.9231  0.7679   
gbc          Gradient Boosting Classifier    0.7481  0.7614  0.9398  0.7599   
xgboost         Extreme Gradient Boosting    0.7438  0.7565  0.9124  0.7681   
ada                  Ada Boost Classifier    0.7412  0.7466  0.9377  0.7548   
rf               Random Forest Classifier    0.7360  0.7384  0.9063  0.7636   
et                 Extra Trees Classifier    0.7359  0.7412  0.8989  0.7669   
lr                    Logistic Regression    0.7127  0.6573  0.9764  0.7180   
lda          Linear Discriminant Analysis    0.7125  0.6555  0.9732  0.7188   
ridge                    Ridge Classifier    0.7114  0.0000  0.9804  0.7157   
dummy                    Dummy Classifier    0.7055  0.5000  1.0000  0.7055   
qda       Quadratic Discriminant Analysis    0.6993  0.6518  0.9131  0.7290   
nb                            Naive Bayes    0.6981 

## If Rules

In [22]:
# create the datasets
df_train = create_dataset(X=df_source, y=df_source_labels, session_list=train, create_features=create_basic_session_features)
df_val = create_dataset(X=df_source, y=df_source_labels, session_list=val, create_features=create_basic_session_features)
df_test = create_dataset(X=df_source, y=df_source_labels, session_list=test, create_features=create_basic_session_features)

  0%|          | 0/7067 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

  0%|          | 0/2356 [00:00<?, ?it/s]

In [23]:
df_train

,question_num,correct
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
127201,14,1
127202,15,0
127203,16,1
127204,17,0


In [24]:
classifier = setup(
    data=df_train,
    test_data=df_val,
    target='correct',
    train_size=1,
    session_id=random_state,
    fix_imbalance=False,
    fix_imbalance_method='RandomOverSampler',
    html=False,
    verbose=True)

                    Description             Value
0                    Session id                51
1                        Target           correct
2                   Target type            Binary
3           Original data shape       (169614, 2)
4        Transformed data shape       (169614, 2)
5   Transformed train set shape       (127206, 2)
6    Transformed test set shape        (42408, 2)
7              Numeric features                 1
8                    Preprocess              True
9               Imputation type            simple
10           Numeric imputation              mean
11       Categorical imputation              mode
12               Fold Generator   StratifiedKFold
13                  Fold Number                10
14                     CPU Jobs                -1
15                      Use GPU             False
16               Log Experiment             False
17              Experiment Name  clf-default-name
18                          USI              db1c


In [25]:
# compare the models
top_model = compare_models(exclude=['knn'], n_select=5)

                                    Model  Accuracy     AUC  Recall   Prec.  \
dt               Decision Tree Classifier    0.7329  0.7296  0.9407  0.7466   
rf               Random Forest Classifier    0.7329  0.7296  0.9407  0.7466   
gbc          Gradient Boosting Classifier    0.7329  0.7291  0.9407  0.7466   
et                 Extra Trees Classifier    0.7329  0.7296  0.9407  0.7466   
xgboost         Extreme Gradient Boosting    0.7329  0.7296  0.9407  0.7466   
lightgbm  Light Gradient Boosting Machine    0.7329  0.7296  0.9407  0.7466   
ada                  Ada Boost Classifier    0.7308  0.7217  0.9786  0.7310   
lr                    Logistic Regression    0.7055  0.5593  1.0000  0.7055   
nb                            Naive Bayes    0.7055  0.6234  1.0000  0.7055   
svm                   SVM - Linear Kernel    0.7055  0.0000  1.0000  0.7055   
ridge                    Ridge Classifier    0.7055  0.0000  1.0000  0.7055   
qda       Quadratic Discriminant Analysis    0.7055 

In [26]:
model = create_model('dt')

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.7304  0.7286  0.9380  0.7455  0.8308  0.2086  0.2469
1       0.7303  0.7242  0.9395  0.7449  0.8309  0.2064  0.2456
2       0.7344  0.7321  0.9418  0.7474  0.8334  0.2191  0.2603
3       0.7339  0.7289  0.9407  0.7474  0.8330  0.2188  0.2591
4       0.7337  0.7329  0.9405  0.7474  0.8329  0.2185  0.2586
5       0.7343  0.7283  0.9423  0.7472  0.8334  0.2180  0.2595
6       0.7338  0.7313  0.9412  0.7472  0.8330  0.2176  0.2583
7       0.7332  0.7295  0.9415  0.7465  0.8327  0.2148  0.2556
8       0.7337  0.7324  0.9413  0.7471  0.8330  0.2172  0.2579
9       0.7318  0.7284  0.9404  0.7458  0.8318  0.2107  0.2507
Mean    0.7329  0.7296  0.9407  0.7466  0.8325  0.2150  0.2552
Std     0.0015  0.0025  0.0012  0.0009  0.0009  0.0044  0.0052


In [27]:
# evaluate against test set
df_predicted = predict_model(estimator=model, data=df_test)
print(classification_report(y_true=df_predicted.correct, y_pred=df_predicted.prediction_label))    

                      Model  Accuracy     AUC  Recall   Prec.      F1   Kappa  \
0  Decision Tree Classifier     0.729  0.7295  0.9405  0.7421  0.8296  0.2108   

      MCC  
0  0.2517  
              precision    recall  f1-score   support

           0       0.62      0.23      0.34     12663
           1       0.74      0.94      0.83     29745

    accuracy                           0.73     42408
   macro avg       0.68      0.59      0.58     42408
weighted avg       0.71      0.73      0.68     42408



In [28]:
df_sample = pd.DataFrame({
    'question_num': list(range(1, 19))
})


model.predict(df_sample)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int8)

In [29]:
dt = create_model('dt', max_leaf_nodes=18)

      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.7304  0.7286  0.9380  0.7455  0.8308  0.2086  0.2469
1       0.7303  0.7242  0.9395  0.7449  0.8309  0.2064  0.2456
2       0.7344  0.7321  0.9418  0.7474  0.8334  0.2191  0.2603
3       0.7339  0.7289  0.9407  0.7474  0.8330  0.2188  0.2591
4       0.7337  0.7329  0.9405  0.7474  0.8329  0.2185  0.2586
5       0.7343  0.7283  0.9423  0.7472  0.8334  0.2180  0.2595
6       0.7338  0.7313  0.9412  0.7472  0.8330  0.2176  0.2583
7       0.7332  0.7295  0.9415  0.7465  0.8327  0.2148  0.2556
8       0.7337  0.7324  0.9413  0.7471  0.8330  0.2172  0.2579
9       0.7318  0.7284  0.9404  0.7458  0.8318  0.2107  0.2507
Mean    0.7329  0.7296  0.9407  0.7466  0.8325  0.2150  0.2552
Std     0.0015  0.0025  0.0012  0.0009  0.0009  0.0044  0.0052


In [30]:
# evaluate agains test set
df_predicted = predict_model(estimator=dt, data=df_test)
print(classification_report(y_true=df_predicted.correct, y_pred=df_predicted.prediction_label))    

                      Model  Accuracy     AUC  Recall   Prec.      F1   Kappa  \
0  Decision Tree Classifier     0.729  0.7295  0.9405  0.7421  0.8296  0.2108   

      MCC  
0  0.2517  
              precision    recall  f1-score   support

           0       0.62      0.23      0.34     12663
           1       0.74      0.94      0.83     29745

    accuracy                           0.73     42408
   macro avg       0.68      0.59      0.58     42408
weighted avg       0.71      0.73      0.68     42408



In [31]:
dt

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=18,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       random_state=51, splitter='best')

In [32]:
from sklearn import tree
from sklearn.tree import _tree

print(tree.export_text(dt))

|--- feature_0 <= 4.50
|   |--- feature_0 <= 1.50
|   |   |--- class: 1
|   |--- feature_0 >  1.50
|   |   |--- feature_0 <= 3.50
|   |   |   |--- feature_0 <= 2.50
|   |   |   |   |--- class: 1
|   |   |   |--- feature_0 >  2.50
|   |   |   |   |--- class: 1
|   |   |--- feature_0 >  3.50
|   |   |   |--- class: 1
|--- feature_0 >  4.50
|   |--- feature_0 <= 17.50
|   |   |--- feature_0 <= 12.50
|   |   |   |--- feature_0 <= 11.50
|   |   |   |   |--- feature_0 <= 9.50
|   |   |   |   |   |--- feature_0 <= 5.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- feature_0 >  5.50
|   |   |   |   |   |   |--- feature_0 <= 7.50
|   |   |   |   |   |   |   |--- feature_0 <= 6.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- feature_0 >  6.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  7.50
|   |   |   |   |   |   |   |--- feature_0 <= 8.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   

In [33]:
# https://mljar.com/blog/extract-rules-decision-tree/

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    feature_names = [f.replace(" ", "_")[:-5] for f in feature_names]
    print("def predict({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "    " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, np.round(threshold,2)))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, np.round(threshold,2)))
            recurse(tree_.children_right[node], depth + 1)
        else:
            #print('####', np.argmax(tree_.value[node][0]))
            #print("{}return {}".format(indent, tree_.value[node]))
            print("{}return {}".format(indent, np.argmax(tree_.value[node][0])))

    recurse(0, 1)

tree_to_code(dt, df_train.columns)

def predict(questio, co):
    if question_num <= 4.5:
        if question_num <= 1.5:
            return 1
        else:  # if question_num > 1.5
            if question_num <= 3.5:
                if question_num <= 2.5:
                    return 1
                else:  # if question_num > 2.5
                    return 1
            else:  # if question_num > 3.5
                return 1
    else:  # if question_num > 4.5
        if question_num <= 17.5:
            if question_num <= 12.5:
                if question_num <= 11.5:
                    if question_num <= 9.5:
                        if question_num <= 5.5:
                            return 1
                        else:  # if question_num > 5.5
                            if question_num <= 7.5:
                                if question_num <= 6.5:
                                    return 1
                                else:  # if question_num > 6.5
                                    return 1
              

In [34]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules

get_rules(dt, df_train.columns, ['incorrect', 'correct'])

['if (question_num > 4.5) and (question_num > 17.5) then class: correct (proba: 95.06%) | based on 7,067 samples',
 'if (question_num > 4.5) and (question_num <= 17.5) and (question_num > 12.5) and (question_num > 13.5) and (question_num > 15.5) and (question_num > 16.5) then class: correct (proba: 69.38%) | based on 7,067 samples',
 'if (question_num <= 4.5) and (question_num > 1.5) and (question_num <= 3.5) and (question_num <= 2.5) then class: correct (proba: 97.89%) | based on 7,067 samples',
 'if (question_num <= 4.5) and (question_num > 1.5) and (question_num <= 3.5) and (question_num > 2.5) then class: correct (proba: 93.29%) | based on 7,067 samples',
 'if (question_num <= 4.5) and (question_num > 1.5) and (question_num > 3.5) then class: correct (proba: 80.2%) | based on 7,067 samples',
 'if (question_num > 4.5) and (question_num <= 17.5) and (question_num <= 12.5) and (question_num <= 11.5) and (question_num <= 9.5) and (question_num <= 5.5) then class: correct (proba: 54.62%

In [35]:
def predict(question_num):
    if question_num <= 4.5:
        if question_num <= 1.5:
            return 1
        else:  # if question_num > 1.5
            if question_num <= 3.5:
                if question_num <= 2.5:
                    return 1
                else:  # if question_num > 2.5
                    return 1
            else:  # if question_num > 3.5
                return 1
    else:  # if question_num > 4.5
        if question_num <= 17.5:
            if question_num <= 12.5:
                if question_num <= 11.5:
                    if question_num <= 9.5:
                        if question_num <= 5.5:
                            return 1
                        else:  # if question_num > 5.5
                            if question_num <= 7.5:
                                if question_num <= 6.5:
                                    return 1
                                else:  # if question_num > 6.5
                                    return 1
                            else:  # if question_num > 7.5
                                if question_num <= 8.5:
                                    return 1
                                else:  # if question_num > 8.5
                                    return 1
                    else:  # if question_num > 9.5
                        if question_num <= 10.5:
                            return 0
                        else:  # if question_num > 10.5
                            return 1
                else:  # if question_num > 11.5
                    return 1
            else:  # if question_num > 12.5
                if question_num <= 13.5:
                    return 0
                else:  # if question_num > 13.5
                    if question_num <= 15.5:
                        if question_num <= 14.5:
                            return 1
                        else:  # if question_num > 14.5
                            return 0
                    else:  # if question_num > 15.5
                        if question_num <= 16.5:
                            return 1
                        else:  # if question_num > 16.5
                            return 1
        else:  # if question_num > 17.5
            return 1
        
# create a prediction mask
mask = []
for question_num in range(1, 19):
    print(question_num, predict(question_num))
    mask.append(predict(question_num))

print(mask)

1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 0
11 1
12 1
13 0
14 1
15 0
16 1
17 1
18 1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]


In [36]:
question_mask = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]

y_true = []
y_pred = []

for session_id in tqdm(test):
    df_session = df_source_labels[df_source_labels['session_id'] == session_id]

    for index, row in df_session.iterrows():
        y_true.append(row['correct'])
        y_pred.append(question_mask[row['question_num']-1])

print(classification_report(y_true, y_pred))




  0%|          | 0/2356 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.58      0.33      0.42     12663
           1       0.76      0.90      0.82     29745

    accuracy                           0.73     42408
   macro avg       0.67      0.61      0.62     42408
weighted avg       0.71      0.73      0.70     42408

